# Exact Computation of Quantile Regression under Random Censoring


Honoré, Khanb, and Powell (2002) defined the estimator for quantile regression under random censoring in (2.15) as

$$
\hat{\beta} \equiv \arg \min _{\beta \in \mathscr{B}} R_n(\beta ; \hat{S}),
$$
where
\begin{align*}
R_n(\beta ; \hat{S}) \equiv & \frac{1}{n} \sum_{i=1}^n \left\{ \left(1-d_i\right) \rho_\pi\left(y_i-\min \left\{x_i^{\prime} \beta, y_i\right\}\right)\right.\\
&\left.+d_i\left[\hat{S}\left(y_i\right)\right]^{-1} \int 1\left\{y_i<c\right\} \rho_\pi\left(y_i-\min \left\{x_i^{\prime} \beta, c\right\}\right) \mathrm{d} \hat{G}(c)\right\}
\end{align*}

Reformulate the estimator to the MIP problem as:
\begin{align*}
\min& \frac{1}{n} \sum_{i=1}^n\left\{\left(1-d_i\right) \left(\pi \cdot sp_i + (1-\pi) \cdot sm_i\right)\right.\\
&\left.+d_i\left[\hat{S}\left(y_i\right)\right]^{-1} \frac{1}{k} \sum_{j=1}^k 1\left\{y_i<a+(j-1)\left(\frac{b-a}{k}\right)\right\} \left(\pi \cdot sp'_{ij} + (1-\pi) \cdot sm'_{ij}\right)\right.\\
&\left.\hat{g}\left(a+(j-1)\left(\frac{b-a}{k}\right)\right) \right\}
\end{align*}

such that
\begin{align}
&y_i - \phi_i + sm_i - sp_i = 0,
&i = 1, \dots, n\\
&y_i - \phi'_{ij} + sm'_{ij} - sp'_{ij} = 0,
&i = 1, \dots, n, \quad j = 1, \dots, k\\
&\phi_i \le x'_i\beta,
&i = 1, \dots, n\\
&\phi_i \le y_i,
&i = 1, \dots, n\\
&\phi_i + \gamma_i \cdot M_i \ge x'_i\beta,
&i = 1, \dots, n\\
&\phi_i + (1-\gamma_i) \cdot M_i\ge y_i,
&i = 1, \dots, n\\
&\phi'_{ij} \le x'_i\beta,
&i = 1, \dots, n, \quad j = 1, \dots, k\\
&\phi'_{ij} \le a+(j-1)\left(\frac{b-a}{k}\right),
&i = 1, \dots, n, \quad j = 1, \dots, k\\
&\phi'_{ij} + \delta_{ij} \cdot N_{ij}\ge x'_i\beta,
&i = 1, \dots, n, \quad j = 1, \dots, k\\
&\phi'_{ij} + (1-\delta_{ij}) \cdot N_{ij}\ge a+(j-1)\left(\frac{b-a}{k}\right),
&i = 1, \dots, n, \quad j = 1, \dots, k\\
&sm_i \ge 0,\quad sp_i \ge 0,\quad sm'_{ij} \ge 0,\quad sp'_{ij} \ge 0,
&i = 1, \dots, n, \quad j = 1, \dots, k\\
&\gamma_i, \delta_{ij} \in \left\{ 0, 1 \right\},
&i = 1, \dots, n, \quad j = 1, \dots, k
\end{align}



In [40]:
import gurobipy as gp
from gurobipy import GRB
import itertools

In [31]:
model = gp.Model("MIP")
x = model.addVars(n, name="x")
y = model.addVars(n, name="y")
beta = model.addVars(2, lb=float('-inf'), name="beta")
phi = model.addVars(n, lb=float('-inf'), name="phi")
phi_prime = model.addVars(n, k, lb=float('-inf'), name="phi_prime")
M = model.addVars(n, name="M")
N = model.addVars(n, k, name="N")
sm = model.addVars(n, name='sm')
sp = model.addVars(n, name='sp')
sm_prime = model.addVars(n, k, name='sm_prime')
sp_prime = model.addVars(n, k, name='sp_prime')
gamma = model.addVars(n, vtype=GRB.BINARY, name='gamma')
delta = model.addVars(n, k, vtype=GRB.BINARY, name='delta')
for i in range(n):
    x[i].lb, x[i].ub, y[i].lb, y[i].ub = X[i], X[i], Y[i], Y[i]

# Monte Carlo Simulation

Similar to Honore, Khan, and Powell (2002), the model used for the Monte Carlo simulation is
\begin{equation*}
    y_i = \min\left\{\beta_0 + x_i\beta_1 + \epsilon_i, c_i \right\},
\end{equation*}

where $x_i$ has a standard normal distribution, the censoring variable $c_i$ is uniformly distributed on the interval $[-1.5, 1.5]$, and the true values $\beta_0$ and $\beta_1$ are -1 and 1, respectively.

I impose four homoskedastic distributions for the error terms $\epsilon_i$ using the standard normal distribution and Student’s-t distributions with 1, 2, and 3 degrees of freedom. I also impose two heteroskedastic distribution for the error terms using $\epsilon_i = \sigma(x_i)\eta_i$, where $\eta_i$ has a standard normal distribution and $\sigma(x_i)\eta_i$ takes a value of $\exp(-x_i)$ and $\exp(x_i)$.


In [44]:
import numpy as np

In [45]:
np.random.seed(123)
n = 100
beta_0 = -1
beta_1 = 1
# x_i ~ Normal(0,1), c_i ~ Uniform(-1.5, 1.5)
X = np.random.normal(loc=0.0, scale=1.0, size=n)
c = np.random.uniform(low=-1.5, high=1.5, size=n)
# calculate generate error terms from six distributions
# Four homoskedastic distributions for error terms. Denote them as e_1, e_2, e_3, e_4, respectively.
# e_k_i ~ N(0,1) and t-distribution with df = 1, 2, 3 for k = 1, 2, 3, 4
e_1 = np.random.normal(loc=0.0, scale=1.0, size=n)
e_2 = np.random.standard_t(df=1, size=n)
e_3 = np.random.standard_t(df=2, size=n)
e_4 = np.random.standard_t(df=3, size=n)
# Two heteroskedastic distributions for error terms. Denote them as e_5, e_6, respectively.
# e_5_i ~ exp(-x_1)*eta_1_i where eta_1_i ~ N(0,1), e_6_i ~ exp(x_1)*eta_2_i where eta_2_i ~ N(0,1)
eta_1 = np.random.normal(loc=0.0, scale=1.0, size=n)
eta_2 = np.random.normal(loc=0.0, scale=1.0, size=n)
e_5 = np.array([np.exp(-1*x_i)*eta_i for x_i, eta_i in zip(X, eta_1)])
e_6 = np.array([np.exp(x_i)*eta_i for x_i, eta_i in zip(X, eta_1)])

In [46]:
# choose error terms from e_1, ..., e_6
e = e_1
# calculate y_i
Y = np.array([min(beta_0+x_i*beta_1+e_i, c_i) for x_i, e_i, c_i in zip(X, e, c)])

In [47]:
# set the number of subinterval k
k = 10
# set the min and max of the censoring points
# TODO: Do we need this?
a = -1.5
b = 1.5

In [48]:
model.addConstrs((y[i]-phi[i]+sm[i]-sp[i] == 0 for i in range(n)))
model.addConstrs((y[i]-phi_prime[(i,j)]+sm_prime[(i,j)]-sp_prime[(i,j)] == 0 for i, j in itertools.product(range(n), range(k))))
model.addConstrs((phi[i] <= beta[0]+beta[1]*x[i] for i in range(n)))
model.addConstrs((phi[i] <= beta[0]+beta[1]*x[i] for i in range(n)))
model.addConstrs((phi[i] <= y[i] for i in range(n)))
model.addConstrs((phi[i] + gamma[i]*M[i] >= beta[0]+beta[1]*x[i] for i in range(n)))
model.addConstrs((phi[i] + (1-gamma[i])*M[i] >= y[i] for i in range(n)))
model.addConstrs((phi_prime[(i,j)]  <= beta[0]+beta[1]*x[i] for i, j in itertools.product(range(n), range(k))))
model.addConstrs((phi_prime[(i,j)]  <= a + (j-1)*(b-a)/k for i, j in itertools.product(range(n), range(k))))
model.addConstrs((phi_prime[(i,j)] + delta[(i,j)]*N[(i,j)] >= beta[0]+beta[1]*x[i] for i, j in itertools.product(range(n), range(k))))
model.addConstrs((phi_prime[(i,j)] + (1-delta[(i,j)])*N[(i,j)] >= a + (j-1)*(b-a)/k for i, j in itertools.product(range(n), range(k))))
model.update()